# Skill estimation using Stan

Let's see how we can use a simple **continuous** variable model to predict a hidden "skill level" of players based on their performance in a collection of games against each other.  We need data (the games and outcomes), and a model of how skill translates into these outcomes (e.g., higher skilled players have a better chance of winning).

In [1]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
%matplotlib inline

We'll use "stan" to define the model; this will compile an inference engine which can then perform Monte Carlo (and some other) approximate inference procedures to give estimates.

We'll describe the (prior) distribution of skill levels as Gaussian, and then model the probability of Player A winning over Player B as a logistic function of the two players' skill differences, with a scaling coefficient (which we can set or try to learn):

In [2]:
skill_model = """
data {
  int<lower=1> N;             # Total number of players
  int<lower=1> E;             # number of games
  real<lower=0> scale;        # scale value for probability computation
  int<lower=0,upper=1> win[E]; # PA wins vs PB
  int PA[E];                  # player info between each game
  int PB[E];                  # 
}
parameters {
  vector [N] skill;           # skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,3); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   # win probability is a logit function of skill difference
}
"""

Now, compile the model.  We'll save a version of it so we don't have to recompile every time.

In [3]:
import pickle
try:     # load it if already compiled
    sm = pickle.load(open('skill_model.pkl', 'rb'))
except:  # ow, compile and save compiled model
    sm = pystan.StanModel(model_code = skill_model)
    with open('skill_model.pkl', 'wb') as f: pickle.dump(sm, f)

We also need the observed data: number of players and games, which pairs played each game, and who won:

In [4]:
# load data
import pandas as pd
train_data=pd.read_csv('train.csv',index_col=False,
        names=['date', 'p1', 'p1_outcome', 'score', 'p2', 'p2_outcome', 'p1_race', 'p2_race', 'addon', 'type'])
#drop other columns for now
train_data.drop(columns=['date','p1_outcome','p2_outcome','p1_race', 'p2_race', 'addon', 'type'], inplace=True)

In [5]:
#train_data = train_data[:10000]
train_data

,p1,score,p2
0,MC,0–2,Stats
1,MC,1–2,Dark
2,MC,0–2,INnoVation
3,MC,0–1,TRUE
4,MC,1–0,Super
...,...,...,...
193069,Keiras,1–2,Harpner
193070,Keiras,2–0,Harpner
193071,Keiras,0–1,maTTzour
193072,Keiras,1–2,nukestrike


In [6]:
# list of players
players = np.unique(np.concatenate((train_data['p1'], train_data['p2'])))

#map between player names to a unique playerid
playerid = dict()
for i in range(len(players)):
    playerid[players[i]]=i+1
    
len(players)

999

In [7]:
# replace player names with player id
train_data['p1'].replace(playerid,inplace=True)
train_data['p2'].replace(playerid,inplace=True)
train_data[['p1_win','p2_win']] = train_data.score.str.split("–",expand=True) 
train_data.drop(columns=['score'],inplace=True)
train_data

,p1,p2,p1_win,p2_win
0,433,753,0,2
1,433,160,1,2
2,433,330,0,2
3,433,782,0,1
4,433,769,1,0
...,...,...,...,...
193069,379,296,1,2
193070,379,296,2,0
193071,379,959,0,1
193072,379,966,1,2


In [8]:
new_train_data=[]
for index,g in train_data.iterrows():
    p1_win, p2_win = int(g['p1_win']),int(g['p2_win'])
    for i in range(p1_win):
        new_train_data.append([g['p1'], g['p2'], 1])
    for i in range(p2_win):
        new_train_data.append([g['p1'], g['p2'], 0])

new_train_data=pd.DataFrame(new_train_data, columns=['p1', 'p2', 'outcome'])


In [9]:
train_data=new_train_data

train_data

,p1,p2,outcome
0,433,753,0
1,433,753,0
2,433,160,1
3,433,160,0
4,433,160,0
...,...,...,...
429888,379,966,0
429889,379,966,0
429890,379,813,1
429891,379,813,0


In [10]:
train_games=np.array([list(r) for r in train_data.to_numpy()])
#p.unique(train_games[:,1])

In [11]:
skill_data = {
    'N': len(players),
    'E': len(train_games),
    'scale': 0.3,
    'win':train_games[:,2],
    'PA': train_games[:,0],
    'PB': train_games[:,1]
}

Now, we can perform MCMC on the model, and extract the samples:

In [12]:
fit = sm.sampling(data=skill_data, iter=1000, chains=4)

In [13]:
samples = fit.extract()

If we just want the mean estimate for each player's skill level, just take the empirical average over the samples:

In [14]:
samples['skill'].mean(0)

array([-1.85968833e+00, -1.17722601e+00, -3.11154356e+00,  1.35054291e+00,
        3.54103772e+00, -1.39433034e+00,  4.75579783e+00,  1.52781578e+00,
        1.58545605e+00, -2.99042860e+00, -2.10101678e+00, -3.61666455e+00,
        6.52596735e-02,  5.40720895e-01, -1.76890181e+00, -1.36017026e-01,
        4.58845762e+00, -6.14894923e+00, -1.83228559e+00,  3.77138288e+00,
        3.60188168e+00, -8.87399755e-01, -8.97096732e-01, -6.21308056e-02,
       -3.12739556e+00, -1.08498045e+00,  4.03440344e+00,  8.75457659e-01,
        9.87697433e-01, -5.34060542e+00,  4.08831275e+00, -1.96120495e-01,
       -7.04168773e+00, -9.62179773e+00,  3.51923172e+00,  2.90423250e+00,
       -1.19846441e+01,  3.23612505e-01,  1.10510778e+00,  5.69050502e-01,
       -6.02172161e-01,  6.19470603e-02, -3.14999969e-01,  3.99697183e+00,
       -1.38412214e+00,  4.92510364e-01, -2.75766257e+00, -3.65429092e+00,
        3.97088993e+00, -2.13521916e+00,  2.50495165e+00,  2.17291867e+00,
        8.22149511e-02,  

If we want to predict which player will win, we might use a direct estimator of that quantity based on the sample values:

In [15]:
# Player 0 vs Player 1 prediction:
def logit(z): return 1./(1.+np.exp(-z))

# Use our model's win probability function (logistic of scaled difference)
#  using the predicted skill difference for each sample:
prob = logit( skill_data['scale']*(samples['skill'][:,0]-samples['skill'][:,1]) ).mean()

print(prob)

0.4494020051979232


# predict

In [16]:
valid_data=pd.read_csv('valid.csv',index_col=False,names=['date', 'p1', 'p1_outcome', 'score', 'p2', 'p2_outcome', 'p1_race', 'p2_race', 'addon', 'type'])

#drop other columns for now
valid_data.drop(columns=['date','score','p2_outcome','p1_race', 'p2_race', 'addon', 'type'], inplace=True)
# replace player names with player id and loser/winner to -1/1
valid_data['p1'].replace(playerid,inplace=True)
valid_data['p2'].replace(playerid,inplace=True)
valid_data['p1_outcome'].replace({"[loser]":-1,"[winner]":+1},inplace=True)

In [27]:
valid_games=[tuple((r[0],r[2],r[1])) for r in valid_data.to_numpy()]
valid_games

[(433, 502, 1),
 (433, 147, -1),
 (433, 370, 1),
 (433, 824, -1),
 (433, 60, -1),
 (433, 824, 1),
 (433, 92, -1),
 (433, 579, 1),
 (433, 361, 1),
 (433, 361, -1),
 (433, 937, -1),
 (433, 329, 1),
 (433, 147, -1),
 (433, 329, 1),
 (433, 207, -1),
 (433, 89, 1),
 (433, 721, -1),
 (433, 579, 1),
 (433, 565, 1),
 (433, 362, 1),
 (433, 263, 1),
 (433, 329, -1),
 (433, 558, -1),
 (433, 89, -1),
 (433, 824, -1),
 (433, 638, -1),
 (433, 473, 1),
 (433, 937, 1),
 (433, 933, -1),
 (433, 977, -1),
 (433, 329, -1),
 (433, 795, -1),
 (433, 420, 1),
 (433, 242, -1),
 (433, 420, 1),
 (433, 721, 1),
 (433, 558, -1),
 (433, 786, -1),
 (433, 351, -1),
 (433, 351, 1),
 (433, 89, 1),
 (433, 933, -1),
 (433, 664, 1),
 (433, 721, -1),
 (433, 795, 1),
 (433, 558, -1),
 (433, 329, -1),
 (433, 329, -1),
 (433, 242, -1),
 (433, 572, -1),
 (433, 207, 1),
 (433, 650, 1),
 (433, 770, -1),
 (433, 853, 1),
 (433, 494, -1),
 (433, 977, 1),
 (433, 242, -1),
 (433, 370, 1),
 (433, 21, -1),
 (433, 991, 1),
 (433, 329, -

In [23]:
num_valid_game=len(valid_games)
acc=0

for g in valid_games:
    try:
        i,j,result=int(g[0]),int(g[1]),int(g[2])
    except:
        num_valid_game-=1
        continue

    prob = logit( skill_data['scale']*(samples['skill'][:,i-1]-samples['skill'][:,j-1]) ).mean()
    if prob > 0.5:
        prob = +1
    else:
        prob = -1
    acc+=(prob==result)
    
print(acc/num_valid_game, num_valid_game)

0.6835661174167881 94007
